## Imports and Data

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

england_matches = pd.read_json('new_data/matches/matches_England.json')
teams = pd.read_json('new_data/teams.json')
england_events = pd.read_json('new_data/events/events_England.json')
players = pd.read_json('new_data/players.json')

## Team Analysis
### Team class for accessing team attributes (id, matches, players, events etc.) 


In [3]:
class team:
    def __init__(self, name:str):
        self.name = name
        self.team_id = self.get_team_id()
        self.matches = self.get_matches()
        self.players = self.get_players()
        self.all_events = self.get_all_events()
    
    def get_team_id(self):
        return int(teams[teams['name'] == self.name]['wyId'])
        
    def get_matches(self, opposition_team=None):
        matches = pd.DataFrame(columns=england_matches.columns)
        specific_matches = pd.DataFrame(columns=england_matches.columns)
        
        for index,row in england_matches.iterrows():
            teams_in_match = [int(team) for team in list(row['teamsData'].keys())]
            if opposition_team is None:
                if self.team_id in teams_in_match:
                    matches = matches.append(row)
            else:
                if self.team_id in teams_in_match and opposition_team.team_id in teams_in_match:
                    specific_matches = specific_matches.append(row)
                
        if opposition_team is None:     
            return matches
        else:
            return specific_matches
    
    def get_players(self):
        return players[players['currentTeamId'] == self.team_id]
    
    def get_all_events(self):
        return england_events[england_events['teamId'] == self.team_id]

## Graphics Generation

In [11]:
from matplotlib.patches import Arc

def draw_pitch():
    fig = plt.figure()
    fig.set_size_inches(14,7)

    ax = plt.subplot(111)
    
    #touchline
    plt.plot([0,0], [-5,105], 'k') #left length
    plt.plot([0,100], [105,105], 'k') #upper width
    plt.plot([100,100], [-5,105], 'k') #right length
    plt.plot([100,0], [-5,-5], 'k') #lower width
    plt.plot([50,50], [-5,105], 'k') #halfway line

    #center
    center_circle = plt.Circle((50,50), radius=7, color='black', fill=False)
    center_spot = plt.Circle((50,50), radius=0.7, color='black', fill=True)
    ax.add_artist(center_circle)
    ax.add_artist(center_spot)

    #left 18 yard box
    plt.plot([0,12], [32,32], 'k')
    plt.plot([0,12], [69,69], 'k')
    plt.plot([12,12], [32,69], 'k')
    left_d = Arc((6, 50), width=18.3, height=20, theta1=310, theta2=50, color='black')
    ax.add_patch(left_d)

    #left 6 yard box
    plt.plot([0,4], [41.25,41.25], 'k')
    plt.plot([0,4], [59.75,59.75], 'k')
    plt.plot([4,4], [59.75,41.25], 'k')

    #right 18 yard box
    plt.plot([100,88], [32,32], 'k')
    plt.plot([100,88], [69,69], 'k')
    plt.plot([88,88], [32,69], 'k')
    right_d = Arc((94, 50), width=18.3, height=20, theta1=130, theta2=230, color='black')
    ax.add_patch(right_d)

    #right 6 yard box
    plt.plot([100,96], [41.25,41.25], 'k')
    plt.plot([100,96], [59.75,59.75], 'k')
    plt.plot([96,96], [59.75,41.25], 'k')

#dataframe should be an 'events' dataframe
def draw_heatmap(dataframe):
    sns.kdeplot(dataframe['x_start'], dataframe['y_start'], shade=True, shade_lowest=False, cmap='Greens', n_levels = 30)
    plt.xlim, plt.ylim = (0,100), (0,100)

    plt.axis('off')
    plt.show()

## Dashboard

In [68]:
from ipywidgets import interact, interactive, interact_manual, widgets
import warnings 
warnings.filterwarnings('ignore')

team_1, team_2 = None, None
matches = None
target_match = None
target_team = None

england_teams = pd.DataFrame(columns=teams.columns)
for index,row in teams.iterrows():
    if row['area']['name'] == 'England':
        england_teams = england_teams.append(row)
team_selector_options = england_teams['name'].tolist()

team_1_selector = widgets.Dropdown(options=team_selector_options, description='Team 1:')
team_2_selector = widgets.Dropdown(options=team_selector_options, description='Team 2:')

def select_teams(team_1_selector, team_2_selector):
    global team_1, team_2, matches
    
    team_1 = team(team_1_selector)
    team_2 = team(team_2_selector)
    matches = team_1.get_matches(team_2)
    
    match_selector = widgets.Dropdown(options=matches['label'].tolist(), description='Match')
    interact(select_match, match_selector=match_selector)
    
def select_match(match_selector) -> pd.DataFrame:
    global matches, target_match, team_1, team_2
    
    target_match = matches[matches['label'] == match_selector]
#     display(target_match)
    target_team_selector = widgets.Dropdown(options=[team_1.name, team_2.name], description='Target Team')
    target_team_selector = interact(select_team, team_selector=target_team_selector)

def select_team(team_selector):
    global team_1, team_2, target_match, target_team
    
    target_team = team_1 if team_selector == team_1.name else team_2
    range_selector = widgets.SelectionRangeSlider(options=[0,45,90], index=(0, 2), description='Interval', disabled=False)
    interact(select_range, range_selector=range_selector)
    
def select_range(range_selector):
    global target_team, target_match
    
    events = target_team.all_events[target_team.all_events['matchId'] == int(target_match['wyId'])]
    events = events[events['eventSec'] >= range_selector[0] * 60]
    events = events[events['eventSec'] <= range_selector[1] * 60]
    passes = events[events['eventName'] == 'Pass']
    passes['x_start'] = passes.apply(lambda row: row['positions'][0]['x'], axis=1)
    passes['y_start'] = passes.apply(lambda row: row['positions'][0]['y'], axis=1)
    
    draw_pitch()
    draw_heatmap(passes)
    
interact_manual(select_teams, team_1_selector=team_1_selector, team_2_selector=team_2_selector)

interactive(children=(Dropdown(description='Team 1:', options=('Newcastle United', 'Huddersfield Town', 'AFC B…

<function __main__.select_teams(team_1_selector, team_2_selector)>

In [59]:
teams.head()

,area,category,city,gender,name,officialName,type,wyId
0,"{'alpha2code': '', 'name': 'England', 'alpha3c...",default,Newcastle upon Tyne,male,Newcastle United,Newcastle United FC,club,1613
1,"{'alpha2code': 'ES', 'name': 'Spain', 'alpha3c...",default,Vigo,male,Celta de Vigo,Real Club Celta de Vigo,club,692
2,"{'alpha2code': 'ES', 'name': 'Spain', 'alpha3c...",default,Barcelona,male,Espanyol,Reial Club Deportiu Espanyol,club,691
3,"{'alpha2code': 'ES', 'name': 'Spain', 'alpha3c...",default,Vitoria-Gasteiz,male,Deportivo Alav\u00e9s,Deportivo Alav\u00e9s,club,696
4,"{'alpha2code': 'ES', 'name': 'Spain', 'alpha3c...",default,Valencia,male,Levante,Levante UD,club,695
